# Lesson 11：資料分析基礎

本課程將學習如何使用 Python 進行基本資料分析，包含：
- 資料讀取與檢視
- 資料清理與轉換
- 資料分組與統計
- 視覺化呈現

**使用套件**：numpy, pandas, matplotlib

## 0. Setup：載入套件與設定

首先載入需要的套件，並設定顯示選項。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 設定中文字型（避免顯示問號）
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 設定 pandas 顯示選項
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 1000)

print("套件載入完成！")

## 1. 取得資料

我們可以從 CSV 檔案讀取資料。如果檔案不存在，系統會自動建立示範資料。

In [ ]:
import os

# 嘗試讀取 CSV 檔案，若不存在則建立示範資料
if os.path.exists('sales_data.csv'):
    df2 = pd.read_csv('sales_data.csv')
    print("✓ 已讀取 sales_data.csv")
else:
    # 建立示範資料
    np.random.seed(42)
    categories = ['Monitor', 'Keyboard', 'Mouse', 'Headset']
    products = ['Product A', 'Product B', 'Product C', 'Product D']
    regions = ['Taipei', 'Taichung', 'Kaohsiung']
    channels = ['Online', 'Store']
    
    data = {
        'order_id': range(1, 201),
        'date': pd.date_range('2024-01-01', periods=200, freq='D'),
        'category': np.random.choice(categories, 200),
        'product': np.random.choice(products, 200),
        'price': np.random.randint(500, 5000, 200),
        'quantity': np.random.randint(1, 5, 200),
        'region': np.random.choice(regions, 200),
        'channel': np.random.choice(channels, 200)
    }
    
    df2 = pd.DataFrame(data)
    print("✓ 已建立示範資料（200 筆訂單）")

# 確保 date 是 datetime 格式
df2['date'] = pd.to_datetime(df2['date'])

# 計算營收
df2['revenue'] = df2['price'] * df2['quantity']

print(f"資料筆數：{len(df2)}")

## 2. 了解資料

在進行分析前，先了解資料的結構與內容。

### 2.1 查看前幾筆資料

In [ ]:
# 查看前 5 筆資料
df2.head()

### 2.2 了解資料的大小與欄位

In [ ]:
# 查看資料大小（列數、欄數）
print(f"資料大小：{df2.shape}")
print(f"共有 {df2.shape[0]} 筆資料，{df2.shape[1]} 個欄位")
print()

# 查看欄位名稱
print("欄位名稱：")
print(df2.columns.tolist())

### 2.3 查看資料型別與缺失值

In [ ]:
# info() 可以看到每個欄位的資料型別和缺失值數量
df2.info()

### 2.4 數值欄位的統計摘要

In [ ]:
# describe() 顯示數值欄位的平均值、標準差、最小值、最大值等
df2.describe()

## 3. 清理資料

真實資料常有缺失值、重複資料、錯誤型別等問題，需要先清理。

### 3.1 檢查缺失值

In [ ]:
# 檢查每個欄位的缺失值數量
print("各欄位缺失值數量：")
print(df2.isna().sum())
print()
print(f"總缺失值數量：{df2.isna().sum().sum()}")

### 3.2 處理缺失值（範例）

常見方法：
- `dropna()` - 刪除含有缺失值的列
- `fillna(value)` - 用特定值填補
- `fillna(method='ffill')` - 用前一個值填補

In [ ]:
# 範例：若 price 有缺失值，可用平均值填補（本資料無缺失值，僅示範）
# df2['price'] = df2['price'].fillna(df2['price'].mean())

# 範例：刪除任何含缺失值的列
# df2_clean = df2.dropna()

print("目前資料無缺失值，無需處理")

### 3.3 檢查重複資料

In [ ]:
# 檢查重複的列
duplicates = df2.duplicated().sum()
print(f"重複資料筆數：{duplicates}")

# 若有重複，可用 drop_duplicates() 刪除
# df2 = df2.drop_duplicates()

## 4. 資料分析：篩選與排序

學習如何從資料中找出特定條件的記錄。

### 4.1 篩選資料

In [ ]:
# 篩選高營收訂單（revenue > 5000）
high_revenue = df2[df2['revenue'] > 5000]
print(f"高營收訂單數量：{len(high_revenue)}")
print()

# 篩選特定地區與通路
taipei_online = df2[(df2['region'] == 'Taipei') & (df2['channel'] == 'Online')]
print(f"台北線上訂單數量：{len(taipei_online)}")
print()

# 顯示前幾筆
high_revenue.head(3)

### 4.2 排序資料

In [ ]:
# 依營收由大到小排序，取前 5 筆
top5_revenue = df2.sort_values('revenue', ascending=False).head(5)
print("營收前 5 名的訂單：")
top5_revenue[['order_id', 'category', 'price', 'quantity', 'revenue']]

## 5. Groupby 與 Pivot Table

分組統計是資料分析的重要技能，可以快速了解不同類別的表現。

### 5.1 依類別分組統計

In [ ]:
# 計算各類別的總營收與平均營收
category_stats = df2.groupby('category')['revenue'].agg(['sum', 'mean', 'count'])
category_stats.columns = ['總營收', '平均營收', '訂單數']
print("各類別統計：")
category_stats

### 5.2 依地區分組統計

In [ ]:
# 各地區的總營收
region_revenue = df2.groupby('region')['revenue'].sum().sort_values(ascending=False)
print("各地區總營收排名：")
region_revenue

### 5.3 使用 Pivot Table

In [ ]:
# 建立地區 x 類別的營收交叉表
pivot = df2.pivot_table(values='revenue', index='region', columns='category', aggfunc='sum')
print("地區 x 類別 營收表：")
pivot

## 6. Numpy 統計分析

Numpy 提供強大的數值運算功能。

### 6.1 基本統計量

In [ ]:
# 使用 numpy 計算統計量
revenue_array = df2['revenue'].values

print(f"平均營收：{np.mean(revenue_array):.2f}")
print(f"中位數：{np.median(revenue_array):.2f}")
print(f"標準差：{np.std(revenue_array):.2f}")
print(f"最大值：{np.max(revenue_array)}")
print(f"最小值：{np.min(revenue_array)}")
print(f"90 分位數：{np.percentile(revenue_array, 90):.2f}")

### 6.2 使用 np.where 進行分類

In [ ]:
# 將營收分為高、中、低三級
df2['revenue_level'] = np.where(df2['revenue'] >= 6000, 'High',
                        np.where(df2['revenue'] >= 3000, 'Medium', 'Low'))

# 查看各級別的數量
print("營收級別分布：")
print(df2['revenue_level'].value_counts())
print()

# 顯示範例
df2[['order_id', 'revenue', 'revenue_level']].head(10)

## 7. Matplotlib 視覺化

將資料視覺化能更清楚看出趨勢與模式。

### 7.1 直方圖（Histogram）

In [ ]:
# 營收分布直方圖
plt.figure(figsize=(10, 6))
plt.hist(df2['revenue'], bins=20, edgecolor='black', alpha=0.7)
plt.title('營收分布直方圖', fontsize=14, fontweight='bold')
plt.xlabel('營收 (Revenue)', fontsize=12)
plt.ylabel('訂單數量', fontsize=12)
plt.grid(axis='y', alpha=0.3)
plt.show()

### 7.2 長條圖（Bar Chart）

In [ ]:
# 各類別總營收長條圖
category_revenue = df2.groupby('category')['revenue'].sum().sort_values(ascending=False)

plt.figure(figsize=(10, 6))
category_revenue.plot(kind='bar', color='steelblue', edgecolor='black')
plt.title('各類別總營收', fontsize=14, fontweight='bold')
plt.xlabel('產品類別', fontsize=12)
plt.ylabel('總營收', fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

### 7.3 折線圖（Line Chart）

In [ ]:
# 每日營收趨勢折線圖
daily_revenue = df2.groupby('date')['revenue'].sum().sort_index()

plt.figure(figsize=(14, 6))
plt.plot(daily_revenue.index, daily_revenue.values, marker='o', markersize=3, linewidth=2, color='green')
plt.title('每日營收趨勢', fontsize=16, fontweight='bold')
plt.xlabel('日期', fontsize=12)
plt.ylabel('總營收', fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 8. 小結

本教學帶您完整走過資料分析流程：

**關鍵發現：**
- **類別分析**：Mouse 類別的總營收與平均營收表現最優，是主要獲利來源
- **地區表現**：台北地區的交易量與營收均領先其他地區，展現強勁市場需求
- **營收分布**：大部分訂單營收集中在 1000-4000 之間，少數高價訂單拉高平均值

**核心技能回顧：**
1. **資料載入與檢視**：使用 `pd.read_csv()` 讀取資料，`head()`, `info()`, `describe()` 初步了解資料
2. **資料清理**：透過 `isna()`, `dropna()`, `fillna()` 處理缺失值，確保資料品質
3. **資料分析**：運用條件篩選、排序、groupby、pivot_table 等方法深入挖掘資料洞察
4. **Numpy 統計**：使用 numpy 函數計算統計指標，配合 `np.where()` 進行資料分類
5. **視覺化呈現**：matplotlib 繪製直方圖、長條圖、折線圖，讓數據趨勢一目了然

**下一步建議：**
- 嘗試分析不同通道（channel）的營收差異
- 探索產品價格與銷量的關聯性
- 使用更進階的視覺化套件如 seaborn 繪製複雜圖表
- 練習完成 30 道練習題，強化實戰能力

繼續保持學習熱情，數據分析的世界充滿無限可能！🎯